In [1]:
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc


ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mdbtools is already the newest version (1.0.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')
import numpy as np

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util


In [11]:

def create_summary_df(file_pattern, years):
    """
    指定されたファイルパターンに一致するファイルからデータを読み込み、
    mean、std、minを集めたDataFrameを作成する。

    Args:
        file_pattern (str): ファイル名のパターン (例: "kjdb_prediction_cache_{}.accdb").
        years (list): 対象とする年のリスト。

    Returns:
        pandas.DataFrame: mean、std、minを集めたDataFrame。
        エラーが発生した場合はNoneを返す。
    """

    all_data = {}
    for year in years:
        try:
            file_path = file_pattern.format(year)
            # アクセスDBからデータを読み込むためには、pyodbcなどのライブラリが必要になります。
            # ここではcsvファイルからの読み込みを想定しています。
            if os.path.exists(file_path.replace("accdb","csv")):
              df = pd.read_csv(file_path.replace("accdb","csv"),index_col=0)
            else:
              print(f"{file_path.replace('accdb','csv')}ファイルが存在しません。処理をスキップします。")
              continue

            all_data[year] = df

        except FileNotFoundError:
            print(f"{file_path}が見つかりません。")
            return None
        except Exception as e:
            print(f"{file_path}の読み込み中にエラーが発生しました: {e}")
            return None

    summary_data = {}
    for stat_name in ['mean', 'std', 'min']:
        stat_df = pd.DataFrame()
        for year, df in all_data.items():
            if stat_name in df.index:
                stat_df[year] = df.loc[stat_name]
            else:
                print(f"{year}のデータに{stat_name}が存在しません。")
        summary_data[stat_name] = stat_df

    return summary_data['mean'], summary_data['std'], summary_data['min']


In [10]:

# 使用例
table_name = 'PredictorVariables'  # テーブル名を指定

file_list = [f'kjdb_prediction_cache_{year}.accdb' for year in range(2000, 2025)]
years = range(2000, 2025)

k_a_util.prepare_pd_on_notebook()
print(len(file_list))

df_statistics=[]
for y in years:
  filepath = f'/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_{y}.accdb'
  df_summary = k_a_util.access_file_statistics(filepath, table_name)
  #df_summary = k_a_util.read_table(filepath, table_name)
  columns_to_drop = [ 'key_program_id','key_horse_id', 'desc_horse_no', 'upd', 'obj_rank']
  df = df_summary.drop(columns=columns_to_drop)
  df_statistics.append(df)

25


In [8]:
# CSVファイルから読み込む場合の例 (3つのファイルを読み込む例に変更)
try:
    summary_df_from_csv = summarize_dataframes(df_statistics, years)
    print(summary_df_from_csv)

except FileNotFoundError as e:
    print(f"\nCSVファイルが見つかりませんでした: {e}")
    print("上記はサンプルデータを用いた実行結果です。")
except pd.errors.EmptyDataError as e:
    print(f"\nCSVファイルが空です: {e}")
    print("上記はサンプルデータを用いた実行結果です。")
except Exception as e:  # その他の例外をキャッチ
    print(f"\nCSVファイルの処理中にエラーが発生しました: {e}")
    print("上記はサンプルデータを用いた実行結果です。")



CSVファイルの処理中にエラーが発生しました: Could not convert string '...' to numeric
上記はサンプルデータを用いた実行結果です。
